In [2]:
!pip uninstall numpy
!pip3 uninstall numpy

# !pip install -U pip

^C
^C


In [1]:
!pip install opencv-python mediapipe sklearn matplotlib numpy tensorflow

In [25]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [26]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [27]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_landmarks(image, results):
    # mp_drawing.draw_landmarks(image, results.facemesh, mp_holistic.FACEMESH) # Draw face connections
    # Draw pose connections
#     mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
#                              mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
#                              mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))
    
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))

    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

In [4]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            
            image, results = mediapipe_detection(frame, holistic)
            
            draw_landmarks(image, results)

            cv2.imshow('Opencv Feed', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()
    except Exception as e:
        print(e)
        cap.release()
        cv2.destroyAllWindows()

In [5]:
cap.release()
cv2.destroyAllWindows()

In [28]:
def extract_keypoints(results):
#     pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
#     face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [13]:
result_test = extract_keypoints(results)

In [14]:
np.save('0', result_test)
np.load('0.npy')

array([ 6.73233092e-01,  8.37354898e-01,  4.73090154e-07,  5.83280027e-01,
        8.00918937e-01, -2.19672415e-02,  4.99728143e-01,  7.06397295e-01,
       -2.82632913e-02,  4.44167525e-01,  6.35556698e-01, -3.46818566e-02,
        3.93653899e-01,  5.94395757e-01, -4.10170853e-02,  5.43846488e-01,
        5.02427995e-01, -1.90499518e-03,  5.01346648e-01,  3.90646100e-01,
       -1.62615664e-02,  4.76512372e-01,  3.22021365e-01, -3.21658291e-02,
        4.58580613e-01,  2.59523183e-01, -4.63598371e-02,  6.03765845e-01,
        4.75746810e-01, -1.22376624e-02,  5.86718678e-01,  3.36395621e-01,
       -2.03121360e-02,  5.78595519e-01,  2.47687966e-01, -3.21630798e-02,
        5.71224272e-01,  1.80282205e-01, -4.34053801e-02,  6.62360072e-01,
        4.85313743e-01, -2.88782716e-02,  6.73040688e-01,  3.54931831e-01,
       -4.41131815e-02,  6.74197078e-01,  2.73972601e-01, -5.71569726e-02,
        6.69443548e-01,  2.04668105e-01, -6.85310662e-02,  7.16035008e-01,
        5.22841811e-01, -

# code to create folders of words to train data for

In [29]:
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
# actions = np.array(['hello', 'thanks', 'iloveyou',])
actions = np.array(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','r','s','u','v','w','x','y'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Code to collect Dataset

In [31]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [17]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

# Preprocess data and create labels & features

In [32]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [33]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'r': 16,
 's': 17,
 'u': 18,
 'v': 19,
 'w': 20,
 'x': 21,
 'y': 22}

In [34]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [35]:
print("Shape of sequences : ", np.array(sequences).shape)
print("Shape of labels : ", np.array(labels).shape)

Shape of sequences :  (690, 30, 126)
Shape of labels :  (690,)


In [36]:
# creating x and y data
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [38]:
# spliting data into xtrain xtest, ytrain and ytest with ration of 5% testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [39]:
print("shape of x", X.shape)
print("shape of y", y.shape)
print("shape of xtrain and ytrain", X_train.shape, y_train.shape)
print("shape of xtest and ytest", X_test.shape, y_test.shape)

shape of x (690, 30, 126)
shape of y (690, 23)
shape of xtrain and ytrain (621, 30, 126) (621, 23)
shape of xtest and ytest (69, 30, 126) (69, 23)


# 7. Build and Train *LSTM* Neural Network

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [41]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [42]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [43]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback])

model.summary()

Epoch 1/1000
20/20 [==============================] - 5s 60ms/step - loss: 3.1311 - categorical_accuracy: 0.0499 0s - loss: 3.1311 - categorical_accuracy: 0.04
Epoch 2/1000
20/20 [==============================] - 1s 63ms/step - loss: 3.1645 - categorical_accuracy: 0.0676
Epoch 3/1000
20/20 [==============================] - 1s 63ms/step - loss: 3.1074 - categorical_accuracy: 0.0692
Epoch 4/1000
20/20 [==============================] - 1s 62ms/step - loss: 3.1669 - categorical_accuracy: 0.0837
Epoch 5/1000
20/20 [==============================] - 1s 58ms/step - loss: 3.1875 - categorical_accuracy: 0.0644
Epoch 6/1000
20/20 [==============================] - 1s 59ms/step - loss: 3.2210 - categorical_accuracy: 0.0757
Epoch 7/1000
20/20 [==============================] - 1s 61ms/step - loss: 3.1092 - categorical_accuracy: 0.0821
Epoch 8/1000
20/20 [==============================] - 1s 63ms/step - loss: 3.0553 - categorical_accuracy: 0.1047
Epoch 9/1000
20/20 [=============================

20/20 [==============================] - 1s 60ms/step - loss: 0.5277 - categorical_accuracy: 0.8084
Epoch 73/1000
20/20 [==============================] - 1s 59ms/step - loss: 0.5395 - categorical_accuracy: 0.7874
Epoch 74/1000
20/20 [==============================] - 1s 59ms/step - loss: 0.5419 - categorical_accuracy: 0.7955
Epoch 75/1000
20/20 [==============================] - 1s 59ms/step - loss: 0.5164 - categorical_accuracy: 0.8229
Epoch 76/1000
20/20 [==============================] - 1s 60ms/step - loss: 0.5196 - categorical_accuracy: 0.8229
Epoch 77/1000
20/20 [==============================] - 1s 59ms/step - loss: 0.5306 - categorical_accuracy: 0.8019
Epoch 78/1000
20/20 [==============================] - 1s 60ms/step - loss: 0.5720 - categorical_accuracy: 0.7939
Epoch 79/1000
20/20 [==============================] - 1s 59ms/step - loss: 0.5311 - categorical_accuracy: 0.8084
Epoch 80/1000
20/20 [==============================] - 1s 61ms/step - loss: 0.7666 - categorical_accur

20/20 [==============================] - 1s 62ms/step - loss: 0.3482 - categorical_accuracy: 0.8680
Epoch 144/1000
20/20 [==============================] - 1s 59ms/step - loss: 0.3672 - categorical_accuracy: 0.8760
Epoch 145/1000
20/20 [==============================] - 1s 60ms/step - loss: 0.2494 - categorical_accuracy: 0.9195
Epoch 146/1000
20/20 [==============================] - 1s 61ms/step - loss: 0.2813 - categorical_accuracy: 0.9034
Epoch 147/1000
20/20 [==============================] - 1s 60ms/step - loss: 0.5088 - categorical_accuracy: 0.8100
Epoch 148/1000
20/20 [==============================] - 1s 62ms/step - loss: 0.3499 - categorical_accuracy: 0.8841
Epoch 149/1000
20/20 [==============================] - 1s 70ms/step - loss: 0.2531 - categorical_accuracy: 0.8969
Epoch 150/1000
20/20 [==============================] - 1s 66ms/step - loss: 0.3503 - categorical_accuracy: 0.8905
Epoch 151/1000
20/20 [==============================] - 1s 60ms/step - loss: 0.2933 - categoric

20/20 [==============================] - 1s 62ms/step - loss: 0.0894 - categorical_accuracy: 0.9630
Epoch 215/1000
20/20 [==============================] - 1s 60ms/step - loss: 0.8022 - categorical_accuracy: 0.7794
Epoch 216/1000
20/20 [==============================] - 1s 60ms/step - loss: 0.3432 - categorical_accuracy: 0.9098
Epoch 217/1000
20/20 [==============================] - 1s 59ms/step - loss: 0.1652 - categorical_accuracy: 0.9533
Epoch 218/1000
20/20 [==============================] - 1s 60ms/step - loss: 0.1756 - categorical_accuracy: 0.9469
Epoch 219/1000
20/20 [==============================] - 1s 60ms/step - loss: 0.1123 - categorical_accuracy: 0.9662
Epoch 220/1000
20/20 [==============================] - 1s 61ms/step - loss: 0.0759 - categorical_accuracy: 0.9694
Epoch 221/1000
20/20 [==============================] - 1s 60ms/step - loss: 0.0718 - categorical_accuracy: 0.9742
Epoch 222/1000
20/20 [==============================] - 1s 61ms/step - loss: 0.1356 - categoric

KeyboardInterrupt: 

In [44]:
actions[np.argmax(y_test[4])]

'i'

# Savinf Model

In [45]:
model.save('action.h5')

In [46]:
model.load_weights('action.h5')

# Evalution using confusion matrix and Accuracy

In [47]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [48]:
yhat = model.predict(X_test)

ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

multilabel_confusion_matrix(ytrue, yhat)


array([[[65,  0],
        [ 1,  3]],

       [[67,  0],
        [ 0,  2]],

       [[67,  0],
        [ 0,  2]],

       [[67,  1],
        [ 0,  1]],

       [[63,  3],
        [ 0,  3]],

       [[64,  1],
        [ 0,  4]],

       [[63,  1],
        [ 3,  2]],

       [[68,  0],
        [ 1,  0]],

       [[66,  0],
        [ 0,  3]],

       [[63,  0],
        [ 0,  6]],

       [[68,  0],
        [ 0,  1]],

       [[64,  2],
        [ 0,  3]],

       [[63,  0],
        [ 4,  2]],

       [[68,  0],
        [ 0,  1]],

       [[66,  1],
        [ 0,  2]],

       [[68,  0],
        [ 0,  1]],

       [[66,  1],
        [ 0,  2]],

       [[67,  1],
        [ 0,  1]],

       [[65,  1],
        [ 0,  3]],

       [[66,  0],
        [ 1,  2]],

       [[64,  0],
        [ 0,  5]],

       [[63,  1],
        [ 3,  2]],

       [[64,  0],
        [ 0,  5]]], dtype=int64)

In [49]:
accuracy_score(ytrue, yhat)

0.8115942028985508

#  Test in Real Time

In [50]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [81]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

NameError: name 'image' is not defined

<Figure size 1296x1296 with 0 Axes>

In [82]:
sequence.reverse()
len(sequence)
sequence.append('def')
sequence.reverse()
sequence[-30:]

AttributeError: 'int' object has no attribute 'reverse'

# testing in real time

In [58]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
#             image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

m
<class 'mediapipe.python.solution_base.SolutionOutputs'>
m
<class 'mediapipe.python.solution_base.SolutionOutputs'>
m
<class 'mediapipe.python.solution_base.SolutionOutputs'>
m
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs

n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
s
<class 'mediapipe.python.solution_base.SolutionOutputs'>
s
<class 'mediapipe.python.solution_base.SolutionOutputs'>
s
<class 'mediapipe.python.solution_base.SolutionOutputs'>
s
<class 'mediapipe.python.solution_base.SolutionOutputs'>
s
<class 'mediapipe.python.solution_base.SolutionOutputs'>
s
<class 'mediapipe.python.solution_base.SolutionOutputs'>
a
<class 'mediapipe.python.solution_base.SolutionOutputs'>
a
<class 'mediapipe.python.solution_base.SolutionOutputs'>
i
<class 'mediapipe.python.solution_base.SolutionOutputs'>
i
<class 'mediapipe.python.solution_base.SolutionOutputs'>
i
<class 'mediapipe.python.solution_base.SolutionOutputs'>
i
<class 'mediapipe.python.solution_base.SolutionOutputs'>
i
<class 'mediapipe.python.solution_base.SolutionOutputs

<class 'mediapipe.python.solution_base.SolutionOutputs'>
x
<class 'mediapipe.python.solution_base.SolutionOutputs'>
x
<class 'mediapipe.python.solution_base.SolutionOutputs'>
x
<class 'mediapipe.python.solution_base.SolutionOutputs'>
x
<class 'mediapipe.python.solution_base.SolutionOutputs'>
x
<class 'mediapipe.python.solution_base.SolutionOutputs'>
x
<class 'mediapipe.python.solution_base.SolutionOutputs'>
x
<class 'mediapipe.python.solution_base.SolutionOutputs'>
x
<class 'mediapipe.python.solution_base.SolutionOutputs'>
x
<class 'mediapipe.python.solution_base.SolutionOutputs'>
x
<class 'mediapipe.python.solution_base.SolutionOutputs'>
x
<class 'mediapipe.python.solution_base.SolutionOutputs'>
x
<class 'mediapipe.python.solution_base.SolutionOutputs'>
x
<class 'mediapipe.python.solution_base.SolutionOutputs'>
x
<class 'mediapipe.python.solution_base.SolutionOutputs'>
l
<class 'mediapipe.python.solution_base.SolutionOutputs'>
l
<class 'mediapipe.python.solution_base.SolutionOutputs'>

<class 'mediapipe.python.solution_base.SolutionOutputs'>
f
<class 'mediapipe.python.solution_base.SolutionOutputs'>
f
<class 'mediapipe.python.solution_base.SolutionOutputs'>
f
<class 'mediapipe.python.solution_base.SolutionOutputs'>
f
<class 'mediapipe.python.solution_base.SolutionOutputs'>
f
<class 'mediapipe.python.solution_base.SolutionOutputs'>
f
<class 'mediapipe.python.solution_base.SolutionOutputs'>
f
<class 'mediapipe.python.solution_base.SolutionOutputs'>
f
<class 'mediapipe.python.solution_base.SolutionOutputs'>
f
<class 'mediapipe.python.solution_base.SolutionOutputs'>
f
<class 'mediapipe.python.solution_base.SolutionOutputs'>
f
<class 'mediapipe.python.solution_base.SolutionOutputs'>
f
<class 'mediapipe.python.solution_base.SolutionOutputs'>
f
<class 'mediapipe.python.solution_base.SolutionOutputs'>
f
<class 'mediapipe.python.solution_base.SolutionOutputs'>
f
<class 'mediapipe.python.solution_base.SolutionOutputs'>
f
<class 'mediapipe.python.solution_base.SolutionOutputs'>

u
<class 'mediapipe.python.solution_base.SolutionOutputs'>
u
<class 'mediapipe.python.solution_base.SolutionOutputs'>
u
<class 'mediapipe.python.solution_base.SolutionOutputs'>
u
<class 'mediapipe.python.solution_base.SolutionOutputs'>
u
<class 'mediapipe.python.solution_base.SolutionOutputs'>
u
<class 'mediapipe.python.solution_base.SolutionOutputs'>
u
<class 'mediapipe.python.solution_base.SolutionOutputs'>
u
<class 'mediapipe.python.solution_base.SolutionOutputs'>
u
<class 'mediapipe.python.solution_base.SolutionOutputs'>
u
<class 'mediapipe.python.solution_base.SolutionOutputs'>
u
<class 'mediapipe.python.solution_base.SolutionOutputs'>
u
<class 'mediapipe.python.solution_base.SolutionOutputs'>
u
<class 'mediapipe.python.solution_base.SolutionOutputs'>
u
<class 'mediapipe.python.solution_base.SolutionOutputs'>
u
<class 'mediapipe.python.solution_base.SolutionOutputs'>
u
<class 'mediapipe.python.solution_base.SolutionOutputs'>
u
<class 'mediapipe.python.solution_base.SolutionOutputs

<class 'mediapipe.python.solution_base.SolutionOutputs'>
j
<class 'mediapipe.python.solution_base.SolutionOutputs'>
j
<class 'mediapipe.python.solution_base.SolutionOutputs'>
j
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
i
<class 'mediapipe.python.solution_base.SolutionOutputs'>
i
<class 'mediapipe.python.solution_base.SolutionOutputs'>
y
<class 'mediapipe.python.solution_base.SolutionOutputs'>
x
<class 'mediapipe.python.solution_base.SolutionOutputs'>

<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>

<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>

n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs'>
n
<class 'mediapipe.python.solution_base.SolutionOutputs

In [56]:
cap.release()
cv2.destroyAllWindows()

In [52]:
res[np.argmax(res)] > threshold

True

In [53]:
(num_sequences,30,258)

NameError: name 'num_sequences' is not defined

In [54]:
model.predict(np.expand_dims(X_test[0], axis=0))

array([[3.2362396e-22, 1.0000000e+00, 2.7795096e-29]], dtype=float32)